In [3]:
# Flight Reservation System in Python

import heapq

class FlightGraph:
    def __init__(self):
        self.graph = {}  # Graph: {airport: [(neighbor, cost)]}

    def add_flight(self, src, dest, cost):
        self.graph.setdefault(src, []).append((dest, cost))
        self.graph.setdefault(dest, [])  # Ensure destination exists in graph

    def get_neighbors(self, airport):
        return self.graph.get(airport, [])

    def dijkstra(self, start, end):
        pq = [(0, start, [])]  # (cost, current, path)
        visited = set()
        
        while pq:
            (cost, current, path) = heapq.heappop(pq)
            if current in visited:
                continue
            path = path + [current]
            if current == end:
                return (cost, path)
            visited.add(current)
            for neighbor, edge_cost in self.get_neighbors(current):
                if neighbor not in visited:
                    heapq.heappush(pq, (cost + edge_cost, neighbor, path))
        return (float('inf'), [])


class BookingSystem:
    def __init__(self):
        self.bookings = {}  # {flight_id: {"seats": int, "passengers": []}}

    def add_flight(self, flight_id, seats):
        self.bookings[flight_id] = {"seats": seats, "passengers": []}

    def book_ticket(self, flight_id, passenger):
        flight = self.bookings.get(flight_id)
        if flight and flight["seats"] > 0:
            flight["passengers"].append(passenger)
            flight["seats"] -= 1
            return f"Booking confirmed for {passenger} on flight {flight_id}."
        return f"Booking failed. No available seats on flight {flight_id}."

    def cancel_ticket(self, flight_id, passenger):
        flight = self.bookings.get(flight_id)
        if flight and passenger in flight["passengers"]:
            flight["passengers"].remove(passenger)
            flight["seats"] += 1
            return f"Booking cancelled for {passenger} on flight {flight_id}."
        return f"Cancellation failed. Passenger not found."


# Demo
if __name__ == "__main__":
    # Initialize system
    fg = FlightGraph()
    bs = BookingSystem()

    # Add flights
    fg.add_flight("BLR", "DEL", 3000)
    fg.add_flight("BLR", "HYD", 1000)
    fg.add_flight("HYD", "DEL", 1500)
    fg.add_flight("DEL", "KOL", 2000)

    # Add bookings
    bs.add_flight("BLR-DEL", 2)
    bs.add_flight("HYD-DEL", 1)

    # Route search
    cost, path = fg.dijkstra("BLR", "KOL")
    print(f"Shortest route from BLR to KOL: {' -> '.join(path)} with cost ₹{cost}")

    # Bookings
    print(bs.book_ticket("BLR-DEL", "Alice"))
    print(bs.book_ticket("BLR-DEL", "Bob"))
    print(bs.book_ticket("BLR-DEL", "Charlie"))  # Should fail

    # Cancellation
    print(bs.cancel_ticket("BLR-DEL", "Bob"))
    print(bs.book_ticket("BLR-DEL", "Charlie"))  # Should succeed now